In [49]:
# !pip --version

In [50]:
# !pip install --upgrade pip

In [51]:
# !git clone https://github.com/kakaobrain/pororo.git

In [52]:
# ls

In [53]:
# cd pororo

In [54]:
# !pip install -e .

In [55]:
# !pip install konlpy

In [56]:
# from google.colab import drive
# drive.mount('gdrive')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
news20 = pd.read_csv("gdrive/My Drive/data/반려동물_2020.01.01_2020.12.31_1.csv")
news21 = pd.read_csv("gdrive/My Drive/data/반려동물_2021.01.01_2021.12.31_1.csv")
news22 = pd.read_csv("gdrive/My Drive/data/반려동물_2022.01.01_2022.12.25_1.csv")

print(news20.shape, news21.shape,news22.shape)

(4000, 1) (4000, 1) (4000, 1)


In [ ]:
news20.rename(columns = {'0':'기사 제목'},inplace=True)
news21.rename(columns = {'0':'기사 제목'},inplace=True)
news22.rename(columns = {'0':'기사 제목'},inplace=True)

In [ ]:
news20.head()

,기사 제목
0,"은평구, 반려동물 문화교실 '댕댕이 스쿨' 운영"
1,새해부터 반려동물 미용사·나무의사도 신중년 고용장려금 받는다
2,반려식물로 코로나 우울 해소
3,"“우리 아이와 반려동물, 함께 생활해도 괜찮을까?”"
4,"[영상]반려동물 식품의 세계화 ""정부지원 받아 글로벌기업으로 우뚝"""


In [ ]:
import re

def preprocessing(text):
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]',' ',text)
    text = re.sub('[\s]+',' ',text)
    text = re.sub("[!\"\$\*\']", ' ', text)
    text = text.lower()
    return text

In [ ]:
from tqdm import tqdm
tqdm.pandas()

news20["기사 제목"] = news20["기사 제목"].progress_map(preprocessing)
news21["기사 제목"] = news21["기사 제목"].progress_map(preprocessing)
news22["기사 제목"] = news22["기사 제목"].progress_map(preprocessing)

100%|██████████| 4000/4000 [00:00<00:00, 92618.12it/s]


In [ ]:
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ["년","에","로","월","일","더","도","제","의"]
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [ ]:
news20["기사 제목"] = news20["기사 제목"].map(remove_stopwords)
news21["기사 제목"] = news21["기사 제목"].map(remove_stopwords)
news22["기사 제목"] = news22["기사 제목"].map(remove_stopwords)

In [ ]:
import scipy as sp
import pandas as pd
import numpy as np

from konlpy.tag import Kkma
from konlpy.tag import Hannanum    
from konlpy.tag import Okt         
from konlpy.tag import *
import nltk
import pickle

from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [47]:
# title_list = news20["기사 제목"].values.tolist()
# title_text = ''
# for each_line in title_list:
#     title_text = title_text + each_line + '\n'
# title_text

In [ ]:
from pororo import Pororo

Pororo.available_tasks()

"Available tasks are ['mrc', 'rc', 'qa', 'question_answering', 'machine_reading_comprehension', 'reading_comprehension', 'sentiment', 'sentiment_analysis', 'nli', 'natural_language_inference', 'inference', 'fill', 'fill_in_blank', 'fib', 'para', 'pi', 'cse', 'contextual_subword_embedding', 'similarity', 'sts', 'semantic_textual_similarity', 'sentence_similarity', 'sentvec', 'sentence_embedding', 'sentence_vector', 'se', 'inflection', 'morphological_inflection', 'g2p', 'grapheme_to_phoneme', 'grapheme_to_phoneme_conversion', 'w2v', 'wordvec', 'word2vec', 'word_vector', 'word_embedding', 'tokenize', 'tokenise', 'tokenization', 'tokenisation', 'tok', 'segmentation', 'seg', 'mt', 'machine_translation', 'translation', 'pos', 'tag', 'pos_tagging', 'tagging', 'const', 'constituency', 'constituency_parsing', 'cp', 'pg', 'collocation', 'collocate', 'col', 'word_translation', 'wt', 'summarization', 'summarisation', 'text_summarization', 'text_summarisation', 'summary', 'gec', 'review', 'review_s

In [ ]:
ocr = Pororo(task='ocr')

# 쇼핑 리뷰 데이터셋 훈련 모델

In [ ]:
sa_shop = Pororo(task='sentiment', model='brainbert.base.ko.shopping', lang='ko')

In [ ]:
news22["기사 제목"][34]

'키울 땐 반려동물 죽으면 폐기물 취급 생각부터 바꿔야'

In [ ]:
sa_shop(news22["기사 제목"][34])

'Negative'

# 네이터 영화 리뷰 데이터셋에서 훈련 모델

In [ ]:
sa_movie = Pororo(task='sentiment', model='brainbert.base.ko.nsmc', lang='ko')

In [ ]:
news21["기사 제목"][8]

'허은아 이재명 반려견 이행복 은 어디에 동물에 대한 애정 없어 '

In [ ]:
sa_movie(news21["기사 제목"][8])

'Negative'

In [ ]:
num = 0
emotion = []

while True:
    print(num)
    
    if num == 4000:
        break

    feel = sa_shop(news22["기사 제목"][num])
    print(feel)
    
    emotion.append(feel)

    num = num + 1
df = pd.DataFrame(emotion)
df.columns = ["emotion"]
df.to_csv('sa_shop_emotion_2022.csv',index = False)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Negative
1501
Negative
1502
Positive
1503
Positive
1504
Negative
1505
Positive
1506
Negative
1507
Positive
1508
Negative
1509
Positive
1510
Positive
1511
Positive
1512
Negative
1513
Positive
1514
Positive
1515
Positive
1516
Positive
1517
Positive
1518
Positive
1519
Positive
1520
Negative
1521
Positive
1522
Positive
1523
Positive
1524
Positive
1525
Negative
1526
Negative
1527
Negative
1528
Positive
1529
Positive
1530
Positive
1531
Negative
1532
Negative
1533
Negative
1534
Positive
1535
Positive
1536
Positive
1537
Negative
1538
Negative
1539
Positive
1540
Positive
1541
Negative
1542
Positive
1543
Negative
1544
Positive
1545
Positive
1546
Negative
1547
Positive
1548
Positive
1549
Positive
1550
Negative
1551
Negative
1552
Negative
1553
Negative
1554
Positive
1555
Positive
1556
Positive
1557
Positive
1558
Negative
1559
Positive
1560
Negative
1561
Positive
1562
Negative
1563
Positive
1564
Negative
1565
Negative
1566
Positive
1567
Negative
1568
Positive
156

In [ ]:
num = 0
emotion = []

while True:
    print(num)
    
    if num == 4000:
        break

    feel = sa_movie(news22["기사 제목"][num])
    print(feel)
    
    emotion.append(feel)

    num = num + 1
df = pd.DataFrame(emotion)
df.columns = ["emotion"]
df.to_csv('sa_movie_emotion_2022.csv',index = False)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Positive
1501
Positive
1502
Positive
1503
Positive
1504
Negative
1505
Negative
1506
Negative
1507
Positive
1508
Positive
1509
Positive
1510
Negative
1511
Positive
1512
Negative
1513
Positive
1514
Negative
1515
Negative
1516
Negative
1517
Negative
1518
Positive
1519
Negative
1520
Negative
1521
Positive
1522
Positive
1523
Positive
1524
Negative
1525
Negative
1526
Negative
1527
Positive
1528
Positive
1529
Negative
1530
Negative
1531
Positive
1532
Negative
1533
Negative
1534
Positive
1535
Positive
1536
Positive
1537
Negative
1538
Negative
1539
Positive
1540
Positive
1541
Negative
1542
Positive
1543
Negative
1544
Negative
1545
Positive
1546
Negative
1547
Positive
1548
Negative
1549
Positive
1550
Positive
1551
Negative
1552
Positive
1553
Negative
1554
Negative
1555
Negative
1556
Negative
1557
Positive
1558
Positive
1559
Positive
1560
Positive
1561
Positive
1562
Negative
1563
Positive
1564
Positive
1565
Negative
1566
Positive
1567
Positive
1568
Negative
156

In [ ]:
shop_2022= pd.read_csv("sa_shop_emotion_2022.csv")
shop_2022.head()

,emotion
0,Positive
1,Positive
2,Negative
3,Positive
4,Positive


In [ ]:
movie_2022 = pd.read_csv("sa_movie_emotion_2022.csv")
movie_2022.head()

,emotion
0,Positive
1,Positive
2,Negative
3,Positive
4,Positive
